In [83]:
import torch
import torch.nn as nn
import time

In [84]:
torch.manual_seed(1)
linear = nn.Linear(3,6)
start = time.time()
for _ in range(10000):
  x = torch.rand(2,3)
  y = linear(x)
end = time.time()
time1 = end - start

In [85]:
y

tensor([[ 0.4387, -0.0831, -0.0409, -0.2576, -0.1743, -0.2095],
        [ 0.6387,  0.4346, -0.2536, -0.3780,  0.0089, -0.2784]],
       grad_fn=<AddmmBackward0>)

In [86]:
time1

0.24122142791748047

In [104]:
class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features, mask = None, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        #torch.manual_seed(0)
        #self.weight = torch.ones(out_features, in_features)  #weights set to 1 to check output, random seed not suitable becuase matric size is different
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.bias = None
        #self.mask = mask

    def forward(self, input):
        #if self.mask is None:
        output = input.matmul(self.weight.t())
        print('output', output)
        # else:
        #   output = input.matmul((self.weight * self.mask).t()) # mask added to skip connections between different modalities, see notes
        if self.bias is not None:
            output += self.bias
        print('output_1',output)
        return output

In [105]:
torch.manual_seed(1)
linear1 = CustomLinear(3,6)
start = time.time()
for _ in range(1):
  x = torch.rand(2,3)
  y = linear1(x)
end = time.time()
time2 = end - start

output tensor([[-1.9988e-06,  5.3568e+22,  1.9616e+31,  4.9962e+22,  1.4013e-43,
          5.7117e-35],
        [-1.9383e-06,  5.6158e+21,  2.0667e+30,  5.4439e+21,  2.0879e-43,
          5.9878e-36]], grad_fn=<MmBackward0>)
output_1 tensor([[-1.9988e-06,  5.3568e+22,  1.9616e+31,  4.9962e+22,  6.1895e-01,
          1.9913e-01],
        [-1.9383e-06,  5.6158e+21,  2.0667e+30,  5.4439e+21,  6.1895e-01,
          1.9913e-01]], grad_fn=<AddBackward0>)


In [106]:
time2

0.005042076110839844

In [103]:
y

tensor([[1.2895e+27, 8.0631e+21, 1.1822e+31, 1.3802e+22, 8.1018e-01, 8.7519e-01],
        [2.5590e+27, 1.3324e+21, 1.2474e+30, 2.4158e+21, 8.1018e-01, 8.7519e-01]],
       grad_fn=<AddBackward0>)

In [91]:
time2/time1

1.2807742182897852

In [115]:
import math
from typing import Any

import torch
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
import torch.nn.functional as F

In [118]:
class Linear2(nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        torch.nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            torch.nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

In [131]:
torch.manual_seed(1)
linear2 = Linear2(3,6)
start = time.time()
for _ in range(10000):
  x = torch.rand(2,3)
  y = linear2(x)
end = time.time()
time3 = end - start

In [132]:
time3

0.23681259155273438